In [24]:
import sqlite3
conn = sqlite3.connect('data.db')  
cursor = conn.cursor()


In [25]:
cursor.execute('''
 
CREATE TABLE NodeSensor (
  id INTEGER PRIMARY KEY,
  id_node INTEGER,
  status_sensor BOOLEAN,
  kondisi_battery BOOLEAN,
  data_lokasi POINT,
  data_kelembapan_tanah_1 INT,
  data_kelembapan_tanah_2 INT,
  data_kemiringan_x INT,
  data_kemiringan_y INT,
  data_kemiringan_z INT,
  status_hujan BOOLEAN
);

''')
conn.commit()


In [26]:
cursor.execute('''
 
CREATE TABLE PreprocessingLog (
  id_preprocessing INTEGER PRIMARY KEY,
  id_node INT,
  timestamp TIMESTAMP,
  perubahan_kemiringan INT,
  perubahan_lokasi INT,
  status_waspada BOOLEAN,
  NodeSensor_id_node INTEGER,
  FOREIGN KEY (NodeSensor_id_node) REFERENCES NodeSensor(id)
);
''')
conn.commit()


In [ ]:
import paho.mqtt.client as mqtt

def on_message(client, userdata, message):
    try:
        conn = sqlite3.connect('pira.db')
        cursor = conn.cursor()

        payload = message.payload.decode()
        topic = message.topic

        cursor.execute("INSERT INTO messages (topic, payload) VALUES (?, ?)", (topic, payload))
        conn.commit()
        conn.close()

        print(f"Pesan baru dari topik {topic}: {payload}")

    except Exception as e:
        print(f"Error: {e}")


client = mqtt.Client()
client.on_message = on_message

#18769 16378
client.connect("0.tcp.ap.ngrok.io", 18769, 60)
client.subscribe("test")

client.loop_forever()

In [51]:
conn = sqlite3.connect('sensors.db')  
cursor.execute("SELECT *  FROM NodeSensor")
data_messages = cursor.fetchall()
for row in data_messages:
    print(row)